In [1]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_cohere import ChatCohere
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage
from langchain_cohere.embeddings import CohereEmbeddings

In [2]:
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    cohere_api_key: str
    
    class Config:
        env_file = "../.env"
        extra = "ignore"
        
        
settings = Settings()

In [3]:
llm = ChatCohere(cohere_api_key = settings.cohere_api_key)
llm

ChatCohere(client=<cohere.client.Client object at 0x0000029E05D85D30>, async_client=<cohere.client.AsyncClient object at 0x0000029E072EEBA0>, model='command-a-03-2025', cohere_api_key=SecretStr('**********'))

### 1. Travel Itinerary Generator
#### Tags: ChatPromptTemplate, SystemMessage
#### Task: Create a chat prompt template where the system message defines you as an “expert travel planner” and the human message provides a location, trip length, and preferences. The model should return a day-by-day itinerary.

In [4]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """ 
         You are an expert travel planner with years of experience designing personalized travel itineraries. 
Your task is to create detailed day-by-day plans for travelers based on their location, trip length, and preferences. 
Each itinerary should include activities, sightseeing suggestions, meals, travel tips, and any local recommendations. 
Be concise but thorough, providing realistic timings and practical advice. 
Always consider the traveler’s interests and preferences when recommending activities.
"""),
        ("human", "{question}")
    ]
)
prompt_template

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template=' \n         You are an expert travel planner with years of experience designing personalized travel itineraries. \nYour task is to create detailed day-by-day plans for travelers based on their location, trip length, and preferences. \nEach itinerary should include activities, sightseeing suggestions, meals, travel tips, and any local recommendations. \nBe concise but thorough, providing realistic timings and practical advice. \nAlways consider the traveler’s interests and preferences when recommending activities.\n'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})])

In [5]:
prompt = prompt_template.invoke("I have planned a long trip to Ooty for 1 days")
prompt

ChatPromptValue(messages=[SystemMessage(content=' \n         You are an expert travel planner with years of experience designing personalized travel itineraries. \nYour task is to create detailed day-by-day plans for travelers based on their location, trip length, and preferences. \nEach itinerary should include activities, sightseeing suggestions, meals, travel tips, and any local recommendations. \nBe concise but thorough, providing realistic timings and practical advice. \nAlways consider the traveler’s interests and preferences when recommending activities.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='I have planned a long trip to Ooty for 1 days', additional_kwargs={}, response_metadata={})])

In [6]:
response: AIMessage = llm.invoke(prompt)
print(response.content)

A one-day trip to Ooty (Udhagamandalam) can be quite rewarding if planned well. Here’s a concise yet comprehensive itinerary tailored for a single day, focusing on key attractions and experiences:

---

### **Ooty 1-Day Itinerary**

#### **Morning (8:00 AM - 12:00 PM)**
1. **Arrival & Breakfast (8:00 AM - 9:00 AM)**  
   - Arrive in Ooty early. Start with a local breakfast at **Chai Pani Junction** or **Quality Restaurant** (try their masala dosa or tea).  
   - *Tip*: Ooty is best explored early to avoid crowds.

2. **Botanical Garden (9:00 AM - 10:30 AM)**  
   - Visit the **Government Botanical Garden**, a must-see with its lush greenery, fossil tree, and flower shows.  
   - *Tip*: Wear comfortable shoes for walking.

3. **Ooty Lake (10:30 AM - 11:30 AM)**  
   - Head to **Ooty Lake** for a peaceful walk or a short boat ride (rowboats/paddle boats available).  
   - *Tip*: Carry a light jacket as it can get chilly.

4. **Doddabetta Peak (11:30 AM - 12:00 PM)**  
   - Drive to **Dod

### 2. Customer Support Bot with Context
#### Tags: ChatPromptTemplate, SystemMessage, MessagesPlaceholder
#### Task: Build a prompt template for a customer support bot where the system message sets the tone (“helpful and polite support agent”), the conversation history is preserved using MessagesPlaceholder, and the human input contains the current customer query.


In [7]:
# prompt_template = ChatPromptTemplate.from_messages(
#     [
#         ("system", """ You are a helpful and polite customer support agent. 
# Your role is to assist users with their questions, issues, or concerns in a clear, professional, and friendly manner. 
# Always provide accurate information, step-by-step guidance if needed, and maintain a courteous tone. 
# If you do not know the answer, politely inform the user and suggest the next steps or resources. give maximum 2 line answer
# """),
#         ("human", "{question}")
#     ]
# )

# prompt = prompt_template.invoke("I can’t track my shipment product")
# response: AIMessage = llm.invoke(prompt)
# print(response.content)

In [8]:
customer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """ You are a helpful and polite customer support agent. 
Your role is to assist users with their questions, issues, or concerns in a clear, professional, and friendly manner. 
Always provide accurate information, step-by-step guidance if needed, and maintain a courteous tone. 
If you do not know the answer, politely inform the user and suggest the next steps or resources.
 """),
        MessagesPlaceholder("history"),
        ("human", "{query}")
    ]
)
customer_prompt

ChatPromptTemplate(input_variables=['history', 'query'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_c

In [9]:
prompt = customer_prompt.invoke(
    {
        "history": [
            ("human", "I can’t track my shipment product"),
            ("ai", "I'm sorry to hear you're having trouble tracking your shipment. Could you please provide the tracking number or order details so I can assist you further?")
        ],
        "query" : "My product is broken"
    }
)

In [10]:
response: AIMessage = llm.invoke(prompt)
print(response.content)

I'm sorry to hear that your product arrived broken. Let’s get this resolved for you as quickly as possible. Here’s what you can do:

1. **Document the Damage**: Take clear photos of the broken product and its packaging, if possible. This will help with the claims process.

2. **Contact Customer Support**: Reach out to the seller or the shipping carrier’s customer service. Provide them with your order number, tracking number, and details about the damage. Most companies have a dedicated process for handling damaged items.

3. **Check the Return/Refund Policy**: Review the seller’s return or refund policy to understand your options. Many companies offer replacements or refunds for damaged items.

4. **File a Claim (if applicable)**: If the item was shipped through a carrier, you may need to file a damage claim with them. They will guide you through the process.

5. **Keep Records**: Save all communication, receipts, and documentation related to the issue.

If you’re unsure who to contact

### 3. Medical Symptom Checker (Educational Purpose)
#### Tags: PromptTemplate, ChatPromptTemplate
#### Task: Create a template where the model acts as a medical knowledge assistant (for educational use only) that takes in symptoms and returns possible causes in layman terms, along with a recommendation to see a doctor.

In [11]:
template = """You are a medical knowledge assistant, and your purpose is for educational use only.
Take the input symptoms: {symptoms}, and return possible causes in simple layman terms.
Always include a recommendation to see a doctor for proper diagnosis and treatment."""

In [12]:
medical_prompt = PromptTemplate.from_template(template)
medical_prompt

PromptTemplate(input_variables=['symptoms'], input_types={}, partial_variables={}, template='You are a medical knowledge assistant, and your purpose is for educational use only.\nTake the input symptoms: {symptoms}, and return possible causes in simple layman terms.\nAlways include a recommendation to see a doctor for proper diagnosis and treatment.')

In [13]:
prompt = medical_prompt.invoke({
    "symptoms" : "Today i have a cough"
})

In [14]:
response = llm.invoke(prompt)
print(response.content)

If you have a cough today, it could be due to several possible causes. Here are some common reasons in simple terms:

1. **Common Cold**: A viral infection that often causes a cough along with a runny or stuffy nose, sneezing, and a sore throat.  
2. **Flu (Influenza)**: Similar to a cold but usually more severe, with symptoms like fever, body aches, and a persistent cough.  
3. **Allergies**: Exposure to allergens like pollen, dust, or pet dander can trigger a cough, especially if you have allergies or asthma.  
4. **Sinus Infection**: When sinuses become inflamed or infected, postnasal drip can irritate your throat and cause coughing.  
5. **Bronchitis**: Inflammation of the bronchial tubes in your lungs, often caused by a virus or bacteria, leading to a persistent cough.  
6. **Asthma**: A chronic condition where airways become inflamed and narrow, causing coughing, wheezing, and shortness of breath.  
7. **Acid Reflux (GERD)**: Stomach acid flowing back into the throat can irritate

#### 4. AI Resume Tailor
##### Tags: ChatPromptTemplate, SystemMessage, MessagesPlaceholder
##### Task: Make a prompt template that takes a user’s resume text, a job description, and past conversation history, and rewrites the resume to highlight relevant skills.

In [15]:
resume_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert resume writer and career advisor. Your job is to tailor resumes for specific job applications while keeping them professional."),
        MessagesPlaceholder("history"),
        ("human", "{resume}")
    ]
)
resume_prompt

ChatPromptTemplate(input_variables=['history', 'resume'], input_types={'history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_

In [16]:
prompt = resume_prompt.invoke(
    {
        "history": [
            ("human", "what is the job description in python"),
            ("ai", """In Python, a "job description" typically refers to a structured representation of a job posting or role, often used in job boards, recruitment platforms, or career websites. It usually includes details like job title, responsibilities, requirements, qualifications, and company information. When working with job descriptions in Python, you might parse, analyze, or generate them using libraries like `BeautifulSoup` for web scraping, `spaCy` or `NLTK` for natural language processing, or `pandas` for data manipulation. For example, you might extract keywords, categorize job roles, or match candidates to job postings based on skills and experience. Python's versatility allows for automating tasks like scraping job boards, analyzing trends in job markets, or building recommendation systems for job seekers.""")
        ],
        "resume" : """Resume text: “Worked as a software engineer at XYZ. Developed apps in Python and Java.
Job description: Looking for a backend engineer with Python, API development, and cloud skills."""
    }
)

In [17]:
response = llm.invoke(prompt)
print(response.content)

Here’s how you can tailor the resume text to align with the job description for a **Backend Engineer** role:

**Tailored Resume Text:**  
"Experienced **Backend Engineer** with a strong background in developing scalable applications using **Python** and **Java**. Designed and implemented robust **APIs** to support seamless integration between systems. Proficient in leveraging **cloud technologies** to optimize application performance and reliability. At XYZ, successfully delivered backend solutions that improved system efficiency by 20%."

**Key Improvements:**  
1. **Highlighted Relevant Skills:** Emphasized Python, API development, and cloud skills, which are explicitly mentioned in the job description.  
2. **Role Alignment:** Changed "Software Engineer" to "Backend Engineer" to directly match the job title.  
3. **Quantifiable Achievement:** Added a measurable outcome to demonstrate impact.  
4. **Keyword Matching:** Ensured the resume includes terms like "backend," "APIs," and "cl

####  5. Storytelling Bot for Kids
##### Tags: PromptTemplate, SystemMessage
##### Task: Build a prompt template for an AI that generates bedtime stories for children, with inputs for theme, age group, and moral lesson.

In [ ]:
template = """
You are a creative story teller who writes engaging, age-appropriate bedtime stories for children.

Inputs: 
- Theme: {theme}
- Age Group: {age_group}
- Moral Lesson: {moral_lesson}

Task:
Write a warm, imaginative, and simple bedtime story based on the theme. 
The story should be suitable for the given age group, easy to understand, and entertaining. 
Make sure the moral lesson is gently conveyed at the end of the story. 
Keep the tone positive, calming, and magical.
"""


In [19]:
story_telling_prompt = PromptTemplate.from_template(template)
story_telling_prompt

PromptTemplate(input_variables=['age_group', 'moral_lesson', 'theme'], input_types={}, partial_variables={}, template='\nYou are a creative storyteller who writes engaging, age-appropriate bedtime stories for children.\n\nInputs: \n- Theme: {theme}\n- Age Group: {age_group}\n- Moral Lesson: {moral_lesson}\n\nTask:\nWrite a warm, imaginative, and simple bedtime story based on the theme. \nThe story should be suitable for the given age group, easy to understand, and entertaining. \nMake sure the moral lesson is gently conveyed at the end of the story. \nKeep the tone positive, calming, and magical.\n')

In [26]:
prompt_template = story_telling_prompt.invoke(
    {
        "theme" : "fairy tail",
        "age_group" : 3,
        "moral_lesson" : "Never act before you think"
    }
)
prompt_template

StringPromptValue(text='\nYou are a creative storyteller who writes engaging, age-appropriate bedtime stories for children.\n\nInputs: \n- Theme: fairy tail\n- Age Group: 3\n- Moral Lesson: Never act before you think\n\nTask:\nWrite a warm, imaginative, and simple bedtime story based on the theme. \nThe story should be suitable for the given age group, easy to understand, and entertaining. \nMake sure the moral lesson is gently conveyed at the end of the story. \nKeep the tone positive, calming, and magical.\n')

In [27]:
response = llm.invoke(prompt_template)
print(response.content)

**The Magical Pond of Patience**

Once upon a time, in a sparkling forest filled with twinkling fireflies and soft, glowing flowers, there lived a little fairy named Lila. Lila had sparkly wings that shimmered like the morning dew and a heart full of curiosity. She loved to explore the forest, but sometimes her excitement made her rush into things without thinking.

One sunny morning, Lila fluttered over to her best friend, a wise old turtle named Mr. Shellington, who lived by the Magical Pond. "Mr. Shellington!" Lila called out, her voice bubbling with excitement. "I heard there’s a special flower at the bottom of the pond that grants wishes! Let’s go get it right now!"

Mr. Shellington slowly turned his head, his kind eyes twinkling. "Now, Lila, remember what I always say: *Think before you leap.* The pond is beautiful, but it has its mysteries. Let’s take a moment to plan."

But Lila was too excited to wait. "There’s no time to think! The flower might disappear!" With a flutter of h

#### 6. Social Media Caption Creator
##### Tags: ChatPromptTemplate
##### Task: Create a chat prompt template that generates catchy Instagram captions based on a product description, tone (funny, motivational, luxury), and audience type.

In [23]:
media_caption_template = ChatPromptTemplate.from_messages(
    [
        ("system", """
        You are a creative social media assistant who specializes in writing catchy Instagram captions. 
Your job is to generate short, engaging, and attention-grabbing captions based on three inputs:
1. Product description
2. Tone (funny, motivational, luxury)
3. Audience type (e.g., teenagers, professionals, parents, travelers)

Guidelines:
- Keep captions concise (under 20 words).
- Use emojis where appropriate to enhance engagement.
- Adapt the style and word choice to match the requested tone and audience.
- Always make the product sound appealing and share-worthy. 
"""),
        ("human", "{input}")
    ]
)

In [29]:
media_prompt = media_caption_template.invoke({
    "input": "write instagram captions in funny way"
})
media_prompt

ChatPromptValue(messages=[SystemMessage(content='\n        You are a creative social media assistant who specializes in writing catchy Instagram captions. \nYour job is to generate short, engaging, and attention-grabbing captions based on three inputs:\n1. Product description\n2. Tone (funny, motivational, luxury)\n3. Audience type (e.g., teenagers, professionals, parents, travelers)\n\nGuidelines:\n- Keep captions concise (under 20 words).\n- Use emojis where appropriate to enhance engagement.\n- Adapt the style and word choice to match the requested tone and audience.\n- Always make the product sound appealing and share-worthy. \n', additional_kwargs={}, response_metadata={}), HumanMessage(content='write instagram captions in funny way', additional_kwargs={}, response_metadata={})])

In [32]:
response = llm.invoke(media_prompt)
print(response.content)

Here are three funny Instagram captions for different products:

1. **Product:** Oversized hoodie
**Caption:** Snuggle season is here! 🍂 This hoodie is basically a hug you can wear. 😏 #CozyAF #HoodieWeather

2. **Product:** Instant coffee packets
**Caption:** Adulting is hard, but this coffee makes it a little less terrible. ☕️ #ButFirstCoffee #InstantGratification

3. **Product:** Colorful socks
**Caption:** My socks are cooler than your personality. 😎🧦 #SockGameStrong #FeetSelfie

Let me know if you'd like me to create more funny captions for specific products!


#### 7. AI Language Tutor
##### Tags: ChatPromptTemplate, SystemMessage, MessagesPlaceholder
##### Task: Create a template where the AI acts as a language teacher, remembers past vocabulary from MessagesPlaceholder, and uses them in follow-up example sentences. 

In [38]:
language_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are an AI language tutor. 
- Teach the user vocabulary and grammar step by step.  
- When the user learns a new word, remember it.  
- Reuse previously learned vocabulary in your examples and exercises.  
- Give short, practical example sentences.  
- Correct the user gently if they make mistakes.  
"""),
        MessagesPlaceholder("history"),
        ("human", "{input}")
    ]
)

In [44]:
prompt = language_prompt.invoke(
    {
        "history": [
            ("human", "what is the apple in hindi"), ("ai", """In Hindi, the word for "apple" is **सेब** (pronounced as "seb").  

Example sentence:  
**मैंने एक सेब खाया।**  
(I ate an apple.)  

Remember:  
- **सेब** (seb) is a masculine noun in Hindi.  
- You can use it in sentences like: **क्या तुम्हें सेब पसंद है?** (Do you like apples?)  

Let me know if you'd like to learn more words or practice further!""")
        ],
        "input": "what is the man in Arabic"
    }
)
prompt

ChatPromptValue(messages=[SystemMessage(content='You are an AI language tutor. \n- Teach the user vocabulary and grammar step by step.  \n- When the user learns a new word, remember it.  \n- Reuse previously learned vocabulary in your examples and exercises.  \n- Give short, practical example sentences.  \n- Correct the user gently if they make mistakes.  \n', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the apple in hindi', additional_kwargs={}, response_metadata={}), AIMessage(content='In Hindi, the word for "apple" is **सेब** (pronounced as "seb").  \n\nExample sentence:  \n**मैंने एक सेब खाया।**  \n(I ate an apple.)  \n\nRemember:  \n- **सेब** (seb) is a masculine noun in Hindi.  \n- You can use it in sentences like: **क्या तुम्हें सेब पसंद है?** (Do you like apples?)  \n\nLet me know if you\'d like to learn more words or practice further!', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the man in Arabic', additional_kwargs

In [45]:
response = llm.invoke(prompt)
print(response.content)

In Arabic, the word for "man" is **رَجُل** (pronounced as "rajul").  

Example sentence:  
**الرَجُلُ واقِفٌ هُناك.**  
(The man is standing over there.)  

Remember:  
- **رَجُل** (rajul) is a masculine noun in Arabic.  
- The definite article **الـ** (al) is added to make it "the man."  

Let me know if you'd like to learn more words or practice further!


### 8. Legal Document Simplifier
#### Tags: PromptTemplate, SystemMessage
#### Task: Create a prompt template where the system message says “You are a legal document simplifier” and the human input is a paragraph of legal text. The output should be plain-language explanation.

In [46]:
template = """You are a legal document simplifier. Your job is to read the following legal text and explain it in plain, simple, everyday language that anyone can understand.
        Legal text:{legal_text} Simplified explanation about human message."""

In [47]:
legal_document_simplifier = PromptTemplate.from_template(template)
legal_document_simplifier

PromptTemplate(input_variables=['legal_text'], input_types={}, partial_variables={}, template='You are a legal document simplifier. Your job is to read the following legal text and explain it in plain, simple, everyday language that anyone can understand.\n        Legal text:{legal_text} Simplified explanation about human message.')

In [51]:
doc_simplifier = legal_document_simplifier.invoke(
    {
        "legal_text": "The party of the first part agrees to deliver the goods to the party of the second part no later than thirty (30) days from the execution of this agreement. In the event of a delay caused by unforeseen circumstances beyond the control of the party of the first part, an extension of up to fifteen (15) days shall be permitted, provided that written notice is given to the party of the second part within five (5) business days of the occurrence of such circumstances."
    }
)
doc_simplifier

StringPromptValue(text='You are a legal document simplifier. Your job is to read the following legal text and explain it in plain, simple, everyday language that anyone can understand.\n        Legal text:The party of the first part agrees to deliver the goods to the party of the second part no later than thirty (30) days from the execution of this agreement. In the event of a delay caused by unforeseen circumstances beyond the control of the party of the first part, an extension of up to fifteen (15) days shall be permitted, provided that written notice is given to the party of the second part within five (5) business days of the occurrence of such circumstances. Simplified explanation about human message.')

In [53]:
response = llm.invoke(doc_simplifier)
print(response.content)

Here’s the simplified explanation in plain, everyday language:

**One person (let’s call them "Party A") agrees to deliver goods to another person ("Party B") within 30 days of signing this agreement.**  

**If something unexpected happens that’s completely out of Party A’s control (like a natural disaster or a supply chain issue), they can get an extra 15 days to deliver the goods. But, Party A has to let Party B know in writing within 5 business days after the problem starts.**  

In short:  
- **Party A delivers goods to Party B within 30 days.**  
- **If there’s an unexpected delay, Party A gets 15 extra days, but they must tell Party B quickly (within 5 business days).**


### 9. News Summarizer with Style
#### Tags: ChatPromptTemplate
#### Task: Build a prompt template that takes a news article and outputs a summary in a chosen style: formal, conversational, or humorous.

In [59]:
news_summarizer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a news summarizer. You will read the following news article and create a concise summary. The summary must be written in the chosen style {style}. Available styles: formal, conversational, humorous."),
        ("human", "{article_text}")
    ]
)
news_summarizer_prompt

ChatPromptTemplate(input_variables=['article_text', 'style'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style'], input_types={}, partial_variables={}, template='You are a news summarizer. You will read the following news article and create a concise summary. The summary must be written in the chosen style {style}. Available styles: formal, conversational, humorous.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['article_text'], input_types={}, partial_variables={}, template='{article_text}'), additional_kwargs={})])

In [60]:
news_summarizer = news_summarizer_prompt.invoke(
    {
        "style": "conversational",
        "article_text": """
Apple has announced the launch of its latest iPhone model, which includes a faster processor,
an improved camera system, and a longer battery life. The device will be available in stores
starting next month, with pre-orders opening this Friday.
"""
    }
)
news_summarizer

ChatPromptValue(messages=[SystemMessage(content='You are a news summarizer. You will read the following news article and create a concise summary. The summary must be written in the chosen style conversational. Available styles: formal, conversational, humorous.', additional_kwargs={}, response_metadata={}), HumanMessage(content='\nApple has announced the launch of its latest iPhone model, which includes a faster processor,\nan improved camera system, and a longer battery life. The device will be available in stores\nstarting next month, with pre-orders opening this Friday.\n', additional_kwargs={}, response_metadata={})])

In [62]:
response: AIMessage = llm.invoke(news_summarizer) 
print(response.content)

Hey, guess what? Apple just dropped the news about their newest iPhone! It’s packing a faster processor, a killer camera upgrade, and a battery that lasts longer than ever. You can snag one starting next month, but if you’re eager, pre-orders kick off this Friday. Time to start saving those pennies!


### 10. Fitness Coach
#### Tags: ChatPromptTemplate, SystemMessage, MessagesPlaceholder
#### Task: Create a prompt template where the AI acts as a personal fitness trainer, remembers the user’s past workout logs via MessagesPlaceholder, and suggests a new workout plan.

In [ ]:
fitness_coach_template = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a certified personal fitness trainer.
- Track the user's workout history and fitness goals. Ensure workouts include variety (strength, cardio, flexibility, recovery).
- Adapt recommendations if the user reports soreness, fatigue, or specific preferences. use 1 lines.
"""),
        MessagesPlaceholder("msg")
        ("human", "{input}")
    ]
)
fitness_coach_template

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are a certified personal fitness trainer.\n- Track the user's workout history and fitness goals. Ensure workouts include variety (strength, cardio, flexibility, recovery).\n- Adapt recommendations if the user reports soreness, fatigue, or specific preferences. use 1 lines.\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [82]:
fitness_coach = fitness_coach_template.invoke(
    {
        "msg": [
            ("human", "what workout should i do today"), 
            ("ai", """ Let’s tailor your workout based on your goals and recent activity. Here’s a balanced plan:  

**Strength & Cardio Combo (45-60 mins):**  
- **Warm-up:** 5 mins dynamic stretches + light jogging.  
- **Strength Circuit (3 rounds):**  
  - Squats (12 reps)  
  - Push-ups (10 reps)  
  - Bent-over rows (12 reps)  
  - Plank (30 secs)  
- **Cardio:** 20 mins moderate intensity (e.g., brisk walk, cycling, or HIIT intervals).  
- **Cool-down:** Stretch hamstrings, quads, chest, and shoulders.  

Let me know your goals or how you’re feeling to adjust! 💪"""),
        ],
        "input": "yesterday I did 20 push-ups, 15 squats, and a 2 km run. What should I do today"
    }
)

In [83]:
response = llm.invoke(fitness_coach)
print(response.content)

Today, focus on **recovery and flexibility** with a **light yoga session (20-30 mins)** or **active stretching** to ease soreness. If you feel energetic, add **10-15 core exercises** (e.g., planks, leg raises) for variety.
